# Projet clustering

Objectif du projet : Utiliser le corpus export_articles_EGC_2004_2018 et faire le bilan de l'évolution de la communauté EGC ces 20 dernières années et tenter d'en prédire l'avenir.

Plus précisément, on va tenter de classer les différents sujets de ces 20 dernières années de manière à identifier des tendances de sujet.

En suivant ces tendances, on aura un pronostic des futurs sujets de l'EGC 2020.

L'évolution du projet est suivable à ces addresses : 
- https://github.com/TGranjon/EGC2020
- https://github.com/TGranjon/EGC2020/projects/1

##  Notes :
J'ai abandonné l'utilisation de TreeTagger pour lemmatiser : j'ai pas réussi à le faire fonctionner. Remplacé par FrenchLefffLemmatizer.

Ainsi que Gensim : la forme de ses corpus est étrange et je pense que c'est une perte de temps à se conformer à leur modèle. Remplacé par SKLearn.

Tout le programme peut être effectué sur un ordi sans GPU au prix d'un plus grand temps de traitement.

### Imports

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from random import randint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.datasets import load_files
from sklearn.cluster import AgglomerativeClustering
import time

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
start_time = time.process_time()

### Ouverture du fichier de données

Le fichier de données original contient d'innombrables erreurs dûes au remplacement hâtif du caractère "fin de ligne" par un caractère vide.

Ainsi, plusieurs mots sont collés les uns aux autres et d'autres contiennent un tiret érroné en leur sein.

#### Fichier de données original.
doc = pd.read_csv("export_articles_EGC_2004_2018.csv", sep='\t')
doc

In [4]:
#### Fichier de données corrigé.
##### La correction va jusqu'à la ligne 733.
doc = pd.read_csv("export_articles_EGC_2004_2018_Copie.csv", sep='\t')
doc

,series,booktitle,year,title,abstract,authors,pdf1page,pdfarticle
0,Revue des Nouvelles Technologies de l'Information,EGC,2018,#Idéo2017 : une plateforme citoyenne dédiée à ...,Cette plateforme a pour objectif de permettre ...,"Claudia Marinica, Julien Longhi, Nader Hassine...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002425
1,Revue des Nouvelles Technologies de l'Information,EGC,2018,A two level co-clustering algorithm for very l...,La classification croisée (co-clustering) est ...,"Marius Barctus, Marc Boullé, Fabrice Clérot",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002372
2,Revue des Nouvelles Technologies de l'Information,EGC,2018,ALGeoSPF: Un modèle de factorisation basé sur ...,La recommandation de points d'intérêts est dev...,"Jean-Benoît Griesner, Talel Abdesssalem, Huber...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002380
3,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse des sentiments à partir des commentair...,L'analyse des sentiments est un processus pend...,"Abdeljalil Elouardighi, Mohcine Maghfour, Hafd...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002397
4,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse en rôles sémantiques pour le résumé au...,Cet article présente une approche visant à ext...,"Elyase Lassouli, Yasmine Mesbahi, Camille Prad...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002384
5,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse Ontologique de scénario dans un contex...,NaN,"Marwan Batrouni, Aurélie Bertaux, Christophe N...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002414
6,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apport de la fouille de données pour la préven...,Avec plus de 800 000 décès par an dans le mond...,"Romain Billot, Sofian Berrouiguet, Mark Larsen...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002376
7,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apport des modèles locaux pour les K-moyennes ...,"Dans le cadre du clustering prédictif, pour at...","Vincent Lemaire, Oumaima Alaoui Ismaili",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002379
8,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apprendre les relations de préférence et de co...,"En classification multi-labels, chaque instanc...","Khalil Laghmari, Christophe Marsala, Mohammed ...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002381
9,Revue des Nouvelles Technologies de l'Information,EGC,2018,Approche contextuelle par régression pour les ...,Les tests A/B sont des procédures utilisées pa...,"Emmanuelle Claeys, Pierre Gançarski, Myriam Ma...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002387


En travaillant avec title et abstract, on va réaliser les transformations.
Tous les documents ont des titres mais ils n'ont pas tous d'abstract.
On va donc utiliser une concaténation du titre et de l'abstract afin d'être sûr d'avoir de l'information.

### Lemmatisation et nettoyage des sujets

Il faut mettre les textes en minuscules, tokeniser, retirer la ponctuation, (retirer les nombres), retirer les mots vide puis lemmatiser.

In [5]:
# Fichier des mots à retirer du texte.
cleaner = open("clean.txt","r", encoding="utf-8")
stops = []
for word in cleaner:
    stops.append(word.replace("\n",""))
cleaner.close()

In [6]:
lemmer = FrenchLefffLemmatizer()

# Lemmatiser le texte.
def lemm_tokens(tokens, lemmer):
    lemmed = []
    for item in tokens:
        lemmed.append(lemmer.lemmatize(item))
    return lemmed

def no_numbers(tokens):
    for t in range(len(tokens)):
        tokens[t] == "".join(i for i in tokens[t] if not i.isdigit())
    for token in tokens:
        if token == "":
            tokens.remove("")

# Retirer la ponctuation et autres symboles.
def punct(text):
    for word in text.split(" "):
        if word.endswith("."):
            text = text.replace(word, word.replace(".",""))
        if word.endswith(","):
            text = text.replace(word, word.replace(",",""))
        if word.endswith("?"):
            text = text.replace(word, word.replace("?",""))
        if word.endswith("!"):
            text = text.replace(word, word.replace("!",""))
        if word.endswith(";"):
            text = text.replace(word, word.replace(";",""))
        if word.endswith(":"):
            text = text.replace(word, word.replace(":",""))
        if word.startswith("("):
            text = text.replace(word, word.replace("(",""))
        if word.endswith(")"):
            text = text.replace(word, word.replace(")",""))
        if word.startswith("'"):
            text = text.replace(word, word.replace("'",""))
        if word.endswith("'"):
            text = text.replace(word, word.replace("'",""))
        if word.startswith("d'"):
            text = text.replace(word, word.replace("d'",""))
        if word.startswith("l'"):
            text = text.replace(word, word.replace("l'",""))
        if word.startswith("d’"):
            text = text.replace(word, word.replace("d’",""))
        if word.startswith("l’"):
            text = text.replace(word, word.replace("l’",""))
    return text

In [7]:
# Effectuer les opérations de pré-traitement du texte.
# Retirer la ponctuation, tokeniser puis lemmatiser.
def tokenize(text):
    text = punct(text)
    #text = text.translate(str.maketrans('','',string.punctuation))
    print(text)
    tokens = nltk.word_tokenize(text)
    #alpha = no_numbers(tokens)
    clean = [word for word in tokens if word not in stops]
    lemms = lemm_tokens(clean,lemmer)
    #stems = stem_tokens(tokens, stemmer)
    return lemms

In [8]:
# On effectue ces opérations sur tous les titres et tous les abstracts.
resume = [tokenize(doc["title"][0].lower())]
abstract = [tokenize(str(doc["abstract"][0].lower()))]
for line in range(len(doc)): # title et abstract ont la même taille.
    if line != 0:
        resume.append(tokenize(doc["title"][line].lower()))
        abstract.append(tokenize(str(doc["abstract"][line]).lower()))

#idéo2017  une plateforme citoyenne dédiée à analyse des tweets lors des événements politiques
cette plateforme a pour objectif de permettre aux citoyens analyser par eux-mêmes les tweets politiques lors événements spécifiques en france pour le cas de élection présidentielle de 2017 #idéo2017 analysait en quasi temps réel les messages des candidats et fournissait leurs principales caractéristiques usage du lexique politique et des comparaisons entre les candidats
a two level co-clustering algorithm for very large data sets
la classification croisée co-clustering) est une technique qui permet extraire la structure sous-jacente existante entre les lignes et les colonnes une table de données sous forme de blocs plusieurs applications utilisent cette technique cependant de nombreux algorithmes de co-clustering actuels ne passent pas à l'échelle une des approches utilisées avec succès est la méthode modl qui optimise un critère de vraisemblance régularisée cependant pour des tailles plus im

elaboration et utilisation une base de connaissances un domaine technique
ce poster rend compte une entreprise élaboration un système de représentation des connaissances pour le domaine géotechnique
élimination des liens inter-langues erronés dans wikipédia
un lien inter-langue dans wikipédia est un lien qui mène un article appartenant à une édition linguistique à un autre article décrivant le même concept dans une autre langue ces liens sont ajoutés manuellement par les utilisateurs de wikipédia et ainsi ils sont susceptibles être erronés dans ce papier,nous proposons une approche pour élimination automatique des liens interlangues le principe de base est que la présence un lien erroné est révélée par existence un chemin de liens inter-langues reliant deux articles appartenant à une même édition linguistique notre approche élimine des liens inter-langues à partir de ceux qui ont un faible score de correction jusqu'à ce qu'il n'y ait plus de chemins entre deux articles une même édition

peerus review un outil de recherche experts scientifiques
nous proposons un outil de recherche experts appliqué au monde académique sur les données générées par entreprise dsrt dans le cadre de son application peerus 1 un utilisateur soumet le titre le résumé et optionnellement les auteurs et le journal de publication un article scientifique et se voit proposerune liste experts, potentiels reviewers de article soumis algorithme de recherche est un système de votes reposant sur un modèle du langage entrainé à partir un ensemble de plusieurs millions articles scientifiques outil est accessible à chacun sous la forme une application web intitulée peerus review 2
perforecast  un outil de prévision de évolution de séries temporelles pour le planning capacitaire
nous présentons perforecast un outil qui vise à automatiser le processus de planning capacitaire en utilisant des données temporelles univariées et des modèles prédictifs configurés automatiquement objectif est anticiper les problème

la date de pose est souvent un facteur principal explication de la dégradationdes conduites assainissement pour les gestionnaires de ces réseaux connaître cette information permet ainsi par utilisation de modèles de détérioration de prédire état de santé actuel des conduites non encore inspectées cette connaissance est primordiale pour prendre des décisions dans un contexte de forte contrainte budgétaire objectif est ainsi de reconstituer ces dates de pose à partir des caractéristiques du patrimoine et de son environnement les données à manipuler présentent plusieurs niveaux de complexité importants leurs sources sont hétérogènes leur volume est important et les informations sur leur étiquetage dates) sont limitées  seulement 24 % du linéaire est connu pour les réseaux assainissement de la métropole de lyon la base de données sous-jacente contient les caractéristiques connues des conduites profil géométrique matériau utilisé etc) dans ce papier nous proposons de mesurer effet et impact

apprentissage espaces prétopologiques dans un cadre multi-instance pour la structuration de données
nous présentons dans cet article une méthode supervisée de structuration en dag un ensemble d'éléments étant donnés une structure cible et un ensemble de relations sur ces éléments il s'agit apprendre un modèle de structuration par combinaison des relations initiales nous formalisons ce problème dans le cadre de la théorie de la prétopologie qui permet atteindre des modèles de structuration complexes nous montrons que la non-idempotence de la fonction adhérence rentre dans le cadre du formalisme de apprentissage supervisé) multi-instance et nous proposons un algorithme apprentissage reposant sur le dénombrement des «sacs» positifs et négatifs plutôt que sur un ensemble apprentissage standard une première expérimentation de cette méthode est présentée dans un cadre applicatif de fouille de textes consistant à apprendre un modèle de structuration taxonomique un ensemble de termes
apprentis

...

exploration interactive de bases de connaissances  un retour expérience
la navigation au sein de bases de connaissances reste un problèmeouvert s'il existe plusieurs paradigmes de visualisation peu de travaux sur lesretours expérience sont disponibles dans le cadre de cet article nous noussommes intéressés aux différents paradigmes de navigation interactive au seinde bases documentaires annotées sémantiquement  accès à la base deconnaissances s'effectuant à travers ontologie du domaine d'application cesparadigmes ont été évalués dans le cadre une application industrielle(mécanique des fluides et échangeurs thermiques en fonction de critèresdéfinis par les utilisateurs analyse des retours expérience1 nous a permisde spécifier et de réaliser un nouveau navigateur dédié à la gestion dedocuments techniques annotés par une ontologie de domaine  le « eye tree »,navigateur de type « polar fisheye view »
extension de algorithme cure aux fouilles de données volumineuses
nan
extraction automatiq

le transit des flux information dans le réseau internet à échellemondiale est régi par des accords commerciaux entre systèmes autonomes accordsqui sont mis en oeuvre via le protocole de routage bgp la négociationde ces accords commerciaux repose implicitement sur une hiérarchie des systèmesautonomes et la position relative de deux systèmes débouche sur un accordde type client/fournisseur un des systèmes le client est nettement mieuxclassé que l'autre le fournisseur et le client paye le fournisseur pour le transitdes flux information) ou sur un accord de type "peering" transit gratuit dutrafic entre les deux systèmes) en dépit de son importance il n'existe pas dehiérarchie officielle de internet les clauses commerciales des accords entresystèmes autonomes ne sont pas nécessairement publiques ni de consensus surla façon établir une telle hiérarchie nous proposons une heuristique simpleinspirée de la notion de "centralité spectrale" issue de analyse des réseaux sociauxpour analyser la pos

la notion de règles entre attributs est très générale allant des règles association en fouille de données aux dépendances fonctionnelles df) en bases de données malgré cette diversité la syntaxe des règles est toujours la même seule leur sémantique diffère pour une sémantique donnée en fonction des propriétés induites des techniques algorithmiques sont mises en oeuvre pour découvrir les règles à partir des données a partir un ensemble de règles il est aussi utile en pratique de raisonner sur ces règles comme cela est le cas par exemple avec les axiomes armstrong pour les dépendances fonctionnelles dans cet article nous proposons un cadre qui permet de s'assurer qu'une sémantique donnée pour les règles est bien-formée ie les axiomes armstrong sont justes et complets pour cette sémantique les propositions faites dans ce papier proviennent du contexte applicatif de analyse de données de biopuces a partir de plusieurs sémantiques pour les données expression de gènes nous montrons comment c

etiq un étiqueteur inductif convivail pour les corpus de spécialité
nan
étude de textes par leur image
nous proposons une méthode automatique de comparaison de textes reposant sur une technique de transformation un texte en une image de taille donnée et analyse à aide des outils de la géométrie fractale nous présentons une application à étude un corpus de 90 textes longs
étude expérimentale de mesures de qualité de règles association
la validation des connaissances extraites un processus ecd par un expert métier nécessite de filtrer ces connaissances pour ce faire de nombreuses mesures ont été proposées chacune répondant à des besoins spécifiques ces mesures présentent des caractéristiques variées et parfois contradictoires qu'il convient alors d'examiner arguant du fait que la sélection des bonnes connaissances passe aussi par utilisation un ensemble de mesures adaptées au contexte nous présentons dans cet article une étude expérimentale de différentes mesures cette étude est mise en 

les sous-ensembles flous peuvent être utilisés pour représenter des valeurs imprécises comme un intervalle aux limites mal définies ils peuvent également servir à expression de préférences dans les critères de sélection de requêtes en bases de données en représentation des connaissances utilisation de hiérarchies de types est largement répandue afin de modéliser les relations existant entre les types objets un domaine donné nous nous intéressons aux sous-ensembles flous dont le domaine de définition est une hiérarchie éléments partiellement ordonnés par la relation "sorte de" que nous appelons ontologie nous introduisons la notion de sous-ensemble flou défini sur une partie de l'ontologie puis sa forme développée définie sur ensemble de l'ontologie que nous appelons extension du sous-ensemble flou des classes équivalence de sous-ensembles flous définis sur une ontologie peuvent être caractérisées par un représentant unique que nous appelons sous-ensemble flou minimal nous concluons par

Le traitment des données est acceptable. Un meilleur traitement pourrait être envisagé.

Par exemple, le traitement des entités nommées peut être intéressant.

Il faut obtenir un seul jeu de données avec les titres et les abstracts pour pouvoir les traiter et éliminer les cas qui n'ont pas d'abstract.

Il est possible que seul le titre suffise. Il est possible que la racinisation ne soit pas nécéssaire.

In [9]:
# On relie titre et abstract, sauf pour ceux qui ne disposent pas du second.
for line in range(len(abstract)):
    if abstract[line] != "nan":
        for element in abstract[line]:
            resume[line].append(element)

### Tf-Idf

Attention : l'étape suivante est longue.

In [10]:
# Calcul du temps passé
dico_time = time.process_time()

In [11]:
# On se crée un dictionnaire avec tous les mots rencontrés.
# dico constitue le vocabulaire connu du système.
dico = []
for line in resume:
    for word in line:
        dico.append(word)
for word in dico:
    if dico.count(word) != 1:
        dico.remove
len(dico)

80576

In [12]:
print("--- %s secondes ---" % round(time.process_time() - dico_time, 2))

--- 767.73 secondes ---


In [13]:
dico = list(set(dico))
len(dico)

12159

In [14]:
dico

['décrira',
 'est-il',
 "qu'une",
 'ensemblistes',
 'one',
 'fouiller',
 'leacock',
 'similairement',
 'd´agents',
 'criminality',
 'principles',
 'pré-calcul',
 'nearness',
 'borner',
 'beconsidered',
 'unordered',
 'révélant',
 'portant',
 'fonctiondes',
 'réflexion',
 'implantation',
 'panoplie',
 'telecom',
 'governments',
 'apporte',
 'effects',
 'empêche',
 "n'avons",
 'grouper',
 'flexible',
 'ungraphe',
 'aln',
 'compromis',
 'strategies',
 'grandeconsommation',
 'porteur',
 'cbor',
 'cabine',
 'pgr',
 "d'implantation",
 'échange',
 'malédiction',
 'probability',
 'graduelles',
 'banalisé',
 'dimère',
 "d'enquête",
 'nousmontrons',
 'rétrospective',
 'preliminary',
 'reconciliates',
 'blockchain',
 'téléologique',
 'biais',
 'juriste',
 'seinde',
 'representation',
 'knn',
 'généralité',
 "processusd'extraction",
 'victoire',
 'eddectivement',
 'supérieur',
 'borgne',
 'empirique',
 'incorporation',
 'établies',
 'eclat',
 'adaptateur',
 'lbsn',
 'krex',
 'représentons',
 'pair

On pourra réduire le vocabulaire en retirant des mots avec peu ou pas d'importance.

Ce nettoyage se passe dans le fichier `clean.txt`.

In [15]:
# Calcul du Tf.
tf = []
for x in range(len(resume)):
    tf.append([])
    for y in range(len(resume[x])):
        tf[x].append(resume[x].count(resume[x][y]))

Attention : l'étape suivante est longue.

In [16]:
df_time = time.process_time()

In [17]:
# Calcul du Df.
df = {}
for x in range(len(dico)):
    sum = 0
    for y in range(len(resume)):
        if dico[x] in resume[y]:
            sum = sum + 1
    df[dico[x]] = sum

In [18]:
print("--- %s secondes ---" % round(time.process_time() - df_time, 2))

--- 119.78 secondes ---


In [19]:
# Calcul de l'Idf
idf = {}
for word in df:
    idf[word] = np.log10(len(resume)/df[word])

In [20]:
# Calcul du Tf-Idf.
tfidf = []
for x in range(len(resume)):
    tfidf.append({})
    for y in range(len(resume[x])):
        tfidf[x][resume[x][y]] = tf[x][y] * idf[resume[x][y]]
len(tfidf)

1269

Il faut trier les Tf-Idf par valeur et récupérer les 5 meilleurs.

In [21]:
best = []
for item in tfidf:
    topic = ""
    trier = sorted(item.items(), key=lambda kv: kv[1])
    trier.reverse()
    if len(item)<5:
        top = len(item)
    else:
        top = 5
    for i in range(top):
        topic = topic + trier[i][0] + " "
    best.append(topic)

In [22]:
# On résume un article (titre + abstract) à partir des 5 mots les plus importants.
best

['idéo2017 citoyen politiques tweets candidat ',
 'co-clustering modl gagne taille passent ',
 'recommandation mobilité factorisation inféré visités ',
 'arabe sentiment commentaire marocain dialectal ',
 'cds résumé texte composant rôle ',
 'big scénario ontologique data contexte ',
 'tentative décès supervisées patient risque ',
 'formés majoritaire vote global obtenus ',
 'label heureux musique morceau co-occurrence ',
 'a/b test procédure stratification contextuel ',
 'dynamics power understanding big human ',
 'incrémentales clustering som collaboratif auto-organisatrices ',
 'mot-clé catégorisation scientifique relation sémantique ',
 'cardiaque similarité quasi-arithmétiques moyenne type ',
 'community complex network in structure ',
 'importance noeud réseau influe différencie ',
 'similarité campagne sémantique vectorielles complémentarité ',
 "temps-réel l'architecture oblige pertinence démontrées ",
 'complétude prescriptives contrainte commande vérifier ',
 'treillis distri

### Calcul du LSA

Le LSA va calculer, pour un document donné, les documents les plus similaires à son sujet. On classe le résultat le plus similaire dans le cluster du document donné. On reproduit cette étape et on arrête quand à un nombre de clusters ou une similarité trop faible.

En gros, le LSA nous fournit une similarité qu'on utilise pour former une classification hiérarchique.

On aura donc des clusters par sujets. On pourra élaborer nos techniques de datation pour déterminer les tendances et, si possible, créer de nouveaux clusters (avec les K-moyennes) plus précis.

Vectorisation

In [23]:
# Le tf-idf ne sert à rien d'autre que de convertir notre liste en un vecteur utilisable.
vectorizer = TfidfVectorizer(min_df =1)

dtm = vectorizer.fit_transform(best)

## Document Term Matrix vers Latent Sementic Analysis
lsa = TruncatedSVD(2, algorithm = 'randomized')

dtm_lsa = lsa.fit_transform(dtm)

dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

Calcul de similarité des Documents

In [24]:
# On calcule la similarité des documents à l'aide du LSA.

similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)

pd.DataFrame(similarity, index=best, columns=best).head(10)

,idéo2017 citoyen politiques tweets candidat,co-clustering modl gagne taille passent,recommandation mobilité factorisation inféré visités,arabe sentiment commentaire marocain dialectal,cds résumé texte composant rôle,big scénario ontologique data contexte,tentative décès supervisées patient risque,formés majoritaire vote global obtenus,label heureux musique morceau co-occurrence,a/b test procédure stratification contextuel,...,flou sous-ensemble appelons défini l'ontologie,uitliation documentaire fondé contenu aide,nextclosure fermé partitionnement itemsets treillis,incomplets classement probabiliste arbre décision,galois treillis supervisée étude basée,appropriation mask mémoires savoir-faire entreprise,voisinage graphe prétraitement voisin proximité,conceptuels graphe connaissance contrainte validation,veille technologique texte rechercher scientifique,entrepôt naturel risque examinons volumineux
idéo2017 citoyen politiques tweets candidat,1.000000,0.989824,0.968987,0.927127,0.723651,0.695330,0.924563,0.722175,0.867517,0.726303,...,0.750095,0.735816,0.842486,0.840003,0.791061,0.760180,0.563010,0.705331,0.831147,0.639846
co-clustering modl gagne taille passent,0.989824,1.000000,0.994290,0.971018,0.814497,0.585986,0.969375,0.813256,0.929470,0.816725,...,0.836568,0.824691,0.910572,0.908664,0.870061,0.844898,0.674884,0.799026,0.901815,0.742692
recommandation mobilité factorisation inféré visités,0.968987,0.994290,1.000000,0.990979,0.871757,0.496168,0.990047,0.870709,0.963529,0.873638,...,0.890255,0.880337,0.949482,0.948031,0.917696,0.897157,0.749776,0.858630,0.942778,0.809910
arabe sentiment commentaire marocain dialectal,0.927127,0.971018,0.990979,1.000000,0.929554,0.375332,0.999977,0.928765,0.990701,0.930967,...,0.943265,0.935967,0.982975,0.982121,0.962661,0.948262,0.831693,0.919582,0.978958,0.881214
cds résumé texte composant rôle,0.723651,0.814497,0.871757,0.929554,1.000000,0.007160,0.932034,0.999998,0.971073,0.999993,...,0.999235,0.999842,0.981471,0.982340,0.994652,0.998515,0.977811,0.999657,0.985229,0.993420
big scénario ontologique data contexte,0.695330,0.585986,0.496168,0.375332,0.007160,1.000000,0.369035,0.005025,0.245730,0.011011,...,0.046272,0.024952,0.198635,0.194131,0.110400,0.061635,-0.202483,-0.019026,0.178292,-0.107411
tentative décès supervisées patient risque,0.924563,0.969375,0.990047,0.999977,0.932034,0.369035,1.000000,0.931258,0.991601,0.933423,...,0.945496,0.938334,0.984199,0.983375,0.964476,0.950394,0.835440,0.922226,0.980320,0.884400
formés majoritaire vote global obtenus,0.722175,0.813256,0.870709,0.928765,0.999998,0.005025,0.931258,1.000000,0.970561,0.999982,...,0.999149,0.999801,0.981059,0.981939,0.994429,0.998396,0.978256,0.999711,0.984861,0.993662
label heureux musique morceau co-occurrence,0.867517,0.929470,0.963529,0.990701,0.971073,0.245730,0.991601,0.970561,1.000000,0.971986,...,0.979671,0.975168,0.998833,0.998601,0.990542,0.982641,0.899503,0.964488,0.997619,0.937337
a/b test procédure stratification contextuel,0.726303,0.816725,0.873638,0.930967,0.999993,0.011011,0.933423,0.999982,0.971986,1.000000,...,0.999378,0.999903,0.982201,0.983054,0.995043,0.998717,0.976997,0.999549,0.985881,0.992972


### Clustering Hiérarchique Ascendant (HAC)

Le vrai point d'intérêt de cet algorithme est le seuil de similarité qui servira de point d'arrêt.

On va répartir toutes les données réparties précédemment en clusters aux sujets similaires. 

In [25]:
# On choisit ici le nombre de clusters à utiliser.
n_clusters = 100

In [26]:
# On applique le HAC pour créer nos clusters.
clusters = AgglomerativeClustering(n_clusters=n_clusters).fit_predict(similarity)

In [27]:
cluster_list = []
for i in range(n_clusters):
    cluster_list.append([])

In [28]:
for i in range(len(clusters)):
    try:
        cluster_list[clusters[i]].append({best[i]:doc["year"][i]})
    except:
        print("Error while filling clusters.")

In [29]:
print([len(x) for x in cluster_list])

[57, 7, 2, 35, 29, 9, 15, 13, 23, 9, 7, 46, 54, 21, 44, 6, 37, 7, 9, 2, 27, 19, 22, 11, 32, 6, 2, 30, 3, 22, 42, 40, 26, 28, 9, 10, 7, 3, 5, 6, 2, 36, 4, 12, 12, 5, 1, 14, 2, 1, 9, 6, 2, 4, 1, 29, 5, 5, 22, 1, 18, 4, 22, 5, 17, 2, 5, 21, 1, 1, 5, 22, 8, 4, 19, 13, 24, 10, 3, 1, 1, 1, 1, 4, 21, 3, 12, 15, 16, 2, 1, 3, 8, 1, 1, 1, 15, 1, 1, 1]


#### Notes
Il va falloir affiner les clusters en les résumant en 5 mots-clés.

### Travail temporel

On va classer les clusters par tendances. C'est à dire que l'on va finir avec cinq classes contenant des clusters.

Ces classes représenteront des sujets actuels ou abandonnés, d'origine récente ou ancienne.

De manière évidente, un sujet qui est récent ou qui parle des tendances actuelles aura plus de chances de se retrouver dans les sujets de l'EGC 2020. A l'inverse, les sujets vieux ou dont on ne parle plus depuis un moment ont peu de chances de revenir.

In [30]:
# Un cluster temporel ne contient que les années qui composent chaque cluster.
# On crée autant de clusters temporel que de clusters crées précédemments.
temporal_list = []
for i in range(n_clusters):
    temporal_list.append([])

In [31]:
# On récupère les valeurs des clusters
n = 0
for cluster in cluster_list:
    for i in range(len(cluster)):
        temporal_list[n].append(list(cluster[i].values()))
    n += 1

In [32]:
# On crée les clusters temporels à partir des dates contenues dans les clusters.
def remplir(c, t):
    for i in range(len(c)):
        t.append(t[i][0])
    t = t[int(len(t)/2):]
    return t

In [33]:
for i in range(n_clusters):
    temporal_list[i] = remplir(cluster_list[i], temporal_list[i])

In [34]:
print([len(x) for x in temporal_list])

[57, 7, 2, 35, 29, 9, 15, 13, 23, 9, 7, 46, 54, 21, 44, 6, 37, 7, 9, 2, 27, 19, 22, 11, 32, 6, 2, 30, 3, 22, 42, 40, 26, 28, 9, 10, 7, 3, 5, 6, 2, 36, 4, 12, 12, 5, 1, 14, 2, 1, 9, 6, 2, 4, 1, 29, 5, 5, 22, 1, 18, 4, 22, 5, 17, 2, 5, 21, 1, 1, 5, 22, 8, 4, 19, 13, 24, 10, 3, 1, 1, 1, 1, 4, 21, 3, 12, 15, 16, 2, 1, 3, 8, 1, 1, 1, 15, 1, 1, 1]


In [35]:
# L'évaluation temporelle utilise trois valeurs :
# La date de l'article le plus vieux du cluster
# La date de l'article le plus récent du cluster
# La date moyenne des articles du cluster
def minYear(t):
    return np.min(t)

def maxYear(t):
    return np.max(t)

def moyYear(t):
    return int(np.mean(t))

In [36]:
# On va classer ces clusters selon la combinaison des trois valeurs précédentes :
# Si minYear proche de maxYear alors récent
# Si maxYear proche de minYear alors ancien
# Si moyYear proche de minYear alors ancien
# Si moyYear proche de maxYear alors récent
# Sinon moyen
recent = []
ancien = []
moyen = []
mort = []
nouveau = []

In [37]:
# On calcule l'appartenance aux tendances selon le schéma décrit précédement.
def classe(c, t):
    if minYear(t) < 2011:
        if maxYear(t) < 2011:
            mort.append(c)
        else:
            if moyYear(t) < 2009:
                ancien.append(c)
            elif moyYear(t) > 2014:
                recent.append(c)
            else:
                moyen.append(c)
    elif minYear(t) > 2011:
        nouveau.append(c)
    else:
        if moyYear(t) < 2011:
            ancien.append(c)
        elif moyYear(t) > 2011:
            recent.append(c)
        else:
            moyen.append(c)

In [38]:
for i in range(n_clusters):
    classe(cluster_list[i], temporal_list[i])

In [39]:
# On affiche la classification de tendances.
print("\nRecent:",recent)
print("\nAncien:",ancien)
print("\nMoyen:",moyen)
print("\nMort:",mort)
print("\nNouveau:",nouveau)


Recent: [[{'big scénario ontologique data contexte ': 2018}, {'long-range influence network social in ': 2018}, {'apps android through text features ': 2017}, {'/owl schema rdf to data ': 2015}, {'drift drifted dataset new sudden ': 2014}, {'logic non-relational relational to deduplication ': 2010}], [{'co-clustering modl gagne taille passent ': 2018}, {'témoignage saisi rédigeant saisie préserver ': 2017}, {'recommandation utilisateur sociale préférence recevant ': 2015}, {'classe new mixture labeling examining ': 2015}, {'pertinence recommandation confusion intrinsèquement natif ': 2014}, {'profil utilisateur court-terme long-terme social ': 2011}], [{'protocole stockage libre cohérence cassandra ': 2016}, {'criterion similarity with two clustering ': 2011}], [{'waves dénotée potable souterrain décuplées ': 2017}, {'exhaustif professionnel guidé recommandation expert ': 2011}, {'naïf bayésien variable classifieur poids ': 2011}]]

Ancien: [[{'label multilabel ... court interactif ':

In [40]:
print("Recent :", len(recent), ", Ancien :", len(ancien), ", Moyen :", len(moyen), ", Mort : ", len(mort), ", Nouveau :", len(nouveau))

Recent : 4 , Ancien : 4 , Moyen : 68 , Mort :  13 , Nouveau : 11


La majorité des clusters sont classés en moyen, c'est cohérent mais une classification plus affinée pourrait être intéressante.

In [41]:
print("--- %s secondes ---" % round(time.process_time() - start_time, 2))

--- 925.44 secondes ---


### Évaluation de la classification

On va maximiser la distance inter cluster et minimiser la distance intra cluster

Distance intra max < Distance inter min

Comparons l'efficacité de la similarité de Jaccard et de la similarité Cosine

#### En l'état actuel, l'évaluation ne fonctionne pas car elle se base sur une ancienne version des clusters.
#### La priorité est d'obtenir de meilleurs clusters donc ces fonctions sont mises de coté en attendant.

In [42]:
def jaccard(vec1, vec2):
    commun = 0
    diff = 0
    for key in vec1.keys():
        if key in vec2.keys():
            commun = commun + 1
        else:
            diff = diff + 1
    if diff == 0:
        return -1
    return commun/diff

def cosine(vec1, vec2):
    v1 = np.array(list(vec1.values()))
    v2 = np.array(list(vec2.values()))
    # La partie qui suit devra être retirée plus tard quand on aura des clusters normalisés.
    if len(vec1) > len(vec2):
        v1 = v1[:len(vec2)]
    else:
        v2 = v2[:len(vec1)]
    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))  

In [43]:
#print("J",jaccard(cluster_list[0],cluster_list[1]))
#print("C",cosine(cluster_list[0],cluster_list[1]))

In [44]:
print("--- %s seconds ---" % round(time.process_time() - start_time, 2))

--- 925.59 seconds ---


### Post-soutenance

Réduire le vocabulaire.
Augmenter les clusters.
Évaluer correctement la classification.